In [1]:
from gcs_dual import DualVertex, PolynomialDualGCS, DualEdge
import numpy as np
from program_options import ProgramOptions, PSD_POLY, FREE_POLY
from pydrake.all import MosekSolver, Point, Hyperellipsoid, Hyperrectangle, HPolyhedron
from proper_gcs_policy import obtain_rollout, plot_rollout

In [2]:
options = ProgramOptions()
options.policy_solver = MosekSolver
options.allow_vertex_revisits = True
options.forward_iteration_limit = 10000
options.use_lookahead_policy = True
options.use_lookahead_with_backtracking_policy = False
options.s_procedure_multiplier_degree_for_linear_inequalities = 0
options.s_procedure_take_product_of_linear_constraints = True

options.value_synthesis_use_robust_mosek_parameters = True
options.value_synthesis_MSK_DPAR_INTPNT_CO_TOL_REL_GAP = 1e-9
options.value_synthesis_MSK_DPAR_INTPNT_CO_TOL_PFEAS = 1e-9
options.value_synthesis_MSK_DPAR_INTPNT_CO_TOL_DFEAS = 1e-9
options.pot_type = PSD_POLY

In [5]:
target_set = Point([1])

gcs = PolynomialDualGCS(options, target_set)

vt = gcs.vertices["target"]
v2 = gcs.AddVertex("v2", Hyperrectangle([0], [1]))
v1 = gcs.AddVertex("v1", Hyperrectangle([0], [1]))
vs = gcs.AddVertex("vs", Point([0]))

cost_function = lambda xl, u, xr, xt: 0.01+ np.sum([(xl[i] - xr[i]) ** 2 for i in range(len(xl))])
cost_function_surr = lambda xl, u, xr, xt: 0.01+ np.sum([(xl[i] - xr[i]) ** 2 for i in range(len(xl))])

es1 = gcs.AddEdge(vs, v1, cost_function, cost_function_surr)
es2 = gcs.AddEdge(vs, v2, cost_function, cost_function_surr)

e12, e21 = gcs.AddBidirectionalEdges(v1, v2, cost_function, cost_function_surr)
e1t = gcs.AddEdge(v1, vt, cost_function, cost_function_surr)
e2t = gcs.AddEdge(v2, vt, cost_function, cost_function_surr)

gcs.MaxCostOverVertex(vs)

gcs.SolvePolicy()




Solve took 0.007s
solve successful!
-0.13968892944439695
SolutionResult.kSolutionFound
Solver is Mosek
time 0.0013270378112792969
rescode 0
solution_status 1


In [6]:
traj, v_path, dt = obtain_rollout(gcs, 1, vs, np.array([0])) # TODO: bad assertion
print([v.name for v in v_path])
print(traj)

['vs', 'v2', 'v1', 'v2', 'v1', 'v2', 'v1', 'v2', 'v1', 'v2', 'v1', 'v2', 'v1', 'v2', 'v1', 'v2', 'target']
[array([0.]), array([0.06249325]), array([0.12498537]), array([0.18748118]), array([0.24998196]), array([0.31248549]), array([0.37499008]), array([0.437495]), array([0.5]), array([0.56250499]), array([0.62500991]), array([0.68751451]), array([0.75001804]), array([0.81251881]), array([0.87501463]), array([0.93750674]), array([1.])]


# simple example just a test

In [6]:
target_set = Point([0])

gcs = PolynomialDualGCS(options, target_set, )

vt = gcs.vertices["target"]
v2 = gcs.AddVertex("v2", Hyperrectangle([-1], [1]))
v1 = gcs.AddVertex("v1", Hyperrectangle([-1], [1]))
vs = gcs.AddVertex("vs", Hyperrectangle([-1], [1]))

cost_function = lambda xl, u, xr, xt: np.sum([(xl[i] - xr[i]) ** 2 for i in range(len(xl))])
cost_function_surr = lambda xl, u, xr, xt: np.sum([(xl[i] - xr[i]) ** 2 for i in range(len(xl))])

es1 = gcs.AddEdge(vs, v1, cost_function, cost_function_surr)
e12 = gcs.AddEdge(v1, v2, cost_function, cost_function_surr)
e2t = gcs.AddEdge(v2, vt, cost_function, cost_function_surr)

gcs.MaxCostOverVertex(vs)

gcs.SolvePolicy()

traj, v_path, dt = obtain_rollout(gcs, 1, vs, np.array([1]))
[v.name for v in v_path]
traj

[1.]
Solve took 0.018s
solve successful!
-0.3333332823237622
SolutionResult.kSolutionFound
Solver is Mosek
time 0.00244903564453125
rescode 0
solution_status 1


[array([1.]), array([0.66668752]), array([0.3333458]), array([-0.])]

Set parameter Username
Academic license - for non-commercial use only - expires 2024-12-01


IndexError: list index out of range